In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all(thread=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

In [ ]:
import mygit as mg
from pathlib import Path
import os
folder=os.path.abspath('{}/..'.format(os.getcwd()))
#folder=Path(folder).parent
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
#gt.pyignore()
#git reset --hard origin/master gt.pull()
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
%%time
from sys import path
from datetime import datetime
import os
path.append(os.path.join(os.getcwd(),'src'))

from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_world_fond_indexes(init_db_manager())

In [ ]:
db.get_table('upload_info')

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

from external_sources import mosex,tradingview
import pandas as pd
a=mosex()
a=tradingview()

In [ ]:
%%time
a=job_sectors(init_db_manager())